In [1]:
%load_ext autoreload
%autoreload 2
from knowme.ingest import NotionIngestor
from knowme.embedder import ChromaEmbedder

In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [3]:
ingestor = NotionIngestor(
    folder_path="/Users/abhinavkashyap/abhi/projects/knowme/abhinav_notion"
)
texts, metadatas = ingestor.ingest()

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)

In [5]:
documents = text_splitter.create_documents(texts=texts, metadatas=metadatas)

In [6]:
print(len(documents))
documents[10].page_content

38


"Let's explore this paper to better understand the claimed advantages and improvements over GPT-3.\n\n## Mixture of Experts\n\n![Screenshot 2023-11-27 at 5.38.08 PM.png](GLaM%20Efficient%20Scaling%20of%20Language%20Models%20with%20Mix%20daa1c17d134249129c43a311d3ff31c5/Screenshot_2023-11-27_at_5.38.08_PM.png)\n\nThis is a transformer block. The major computational bottleneck in Transformers is the FFN or the Feedforward Neural Network Component. Instead of a single FFN layer, can we replace it with multiple FFN modules? We can. These individual FFN modules are called experts. Doing this can have multiple advantages:"

In [35]:
from langchain_openai import OpenAIEmbeddings

embedding_function = OpenAIEmbeddings()
store = ChromaEmbedder(
    text_splitter,
    embedding_function=embedding_function,
    embedding_store_directory="./website_vectorstore",
)

In [36]:
chromastore = store.store_embeddings(texts, metadatas)

In [9]:
retriever = chromastore.as_retriever(search_type="similarity", search_kwargs={"k": 6})
retrieved_docs = retriever.invoke("What does Abhinav work as?")

In [12]:
# Some of the information is there in the first retrieved document
# I might have to do some cleaning on the ingestion side
# Remove the links in the page that contain embeddable content
# Consider only those that only contain text.
# This might require a Markdown parser to filter content out
retrieved_docs[1].page_content

'[https://notaku.so/iframes/clickable-image/eyJpbWFnZVVybCI6Imh0dHBzOi8vZW5jcnlwdGVkLXRibjAuZ3N0YXRpYy5jb20vaW1hZ2VzP3E9dGJuOkFOZDlHY1RMV0FlbWMzMFZRZkluMEFuU3AybDE1dUR2WXptYUZfdU1YZyZ1c3FwPUNBVSIsImxpbmtIcmVmIjoiaHR0cHM6Ly90d2l0dGVyLmNvbS9hYmthc2h5YXA5MiJ9](https://notaku.so/iframes/clickable-image/eyJpbWFnZVVybCI6Imh0dHBzOi8vZW5jcnlwdGVkLXRibjAuZ3N0YXRpYy5jb20vaW1hZ2VzP3E9dGJuOkFOZDlHY1RMV0FlbWMzMFZRZkluMEFuU3AybDE1dUR2WXptYUZfdU1YZyZ1c3FwPUNBVSIsImxpbmtIcmVmIjoiaHR0cHM6Ly90d2l0dGVyLmNvbS9hYmthc2h5YXA5MiJ9)\n\nHey I’m Abhinav. I am a AI Scientist at ASUS-AICS. I work on Natural Language Processing and I am  interested in processing clinical notes.'

In [13]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4")

In [15]:
from knowme.knowme_chat import KnowmeChat

chat = KnowmeChat(llm, chromastore)

In [18]:
answer = chat.chat("Where is Abhinav working?", session_id="abc")

In [19]:
answer

'Abhinav is currently working as an AI Scientist at ASUS-AICS in Singapore. His work focuses on Natural Language Processing, particularly processing clinical notes.'

In [22]:
answer = chat.chat("Who did Abhinav work with during PhD?", session_id="abc")
print(answer)

During his PhD, Abhinav worked with Professor Min-Yen Kan. They collaborated on understanding text from various sources, not just mainstream media.


In [23]:
answer = chat.chat("What is the news from Abhinav from March 2024", session_id="abc")

In [24]:
answer

'In March 2024, Abhinav presented a survey paper on sentence representations at the European Chapter of the Association for Computational Linguistics (EACL).'

In [26]:
print(chat.chat("Is there any other news from him for March 2024", session_id="abc"))

No, there is no other news mentioned for Abhinav in March 2024. The only event listed is his presentation of a survey paper on sentence representations at EACL.


In [32]:
print(
    chat.chat(
        "In his blog post on GLaM, what are the three things he mentions?",
        session_id="abc",
    )
)

In his blog post on GLaM, Abhinav mentions three key points:

1. GLaM is the first model to incorporate a mixture of experts into the decoder model, which allows it to perform better than GPT-3 with half the number of parameters for inference. 
2. The GLaM paper introduces a new dataset, which contains 1.6 trillion tokens from the web, to increase the diversity of the data.
3. There are challenges in training mixture of models, such as deciding which expert handles which tokens (also known as routing), and determining the number of experts to add. These decisions often require experimentation.


In [34]:
print(
    chat.chat(
        "What are the titles of the different articles that Abhinav has written?",
        session_id="abc",
    )
)

The titles of the articles written by Abhinav as mentioned in the context are:

1. "GLaM: Efficient Scaling of Language Models with Mixture of Experts"
2. "Publicly Shareable Clinical Large Language model Built on Synthetic Clinical Notes"
3. "MAUVE - Measuring the Gap Between Neural text and Human Text using Divergence Frontiers"


In [ ]:
from langchain_chroma import Chroma

store_from_directory = Chroma(
    persist_directory="./website_vector_store", embedding_function=embedding_function
)